# Generate Dataset

## Music Files

In [1]:
from music21 import stream, note, duration, pitch, metadata, clef
import os
import random
import warnings
from music21.musicxml import m21ToXml

# Suppress annoying MusicXMLWarning
warnings.filterwarnings("ignore", category=m21ToXml.MusicXMLWarning)


def generate_random_note():
    '''
    A function to generate a random note based on a predefined list
    of pitches and durations.
    '''

    # Define a list of possible pitches and durations
    pitches = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'D5', 'E5', 'F5', 'G5']
    durations = ['whole', 'half', 'quarter', 'eighth', '16th']

    # Select a random pitch and duration
    selected_pitch = random.choice(pitches)
    selected_duration = random.choice(durations)

    # Create and return a music21 note
    n = note.Note()
    n.pitch = pitch.Pitch(selected_pitch)
    n.duration = duration.Duration(selected_duration)
    return n, selected_pitch, selected_duration

def generate_synthetic_single_musicxml(num_samples=10, output_folder='../raw_data/musicxml_files'):
    '''
    A function to create a folder of MusicXML files.
    '''
    # check for output - Use this for .py
    # output_folder = os.path.join(os.path.dirname(__file__), os.pardir, 'musicxml_files')
    # if not os.path.exists(output_folder):
    #     os.makedirs(output_folder)

    # for our notebook since __file__ seems to not work
    current_dir = os.getcwd()
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    synthetic_data = []

    for i in range(num_samples):
        """
        to generate a file with a single note
        """
        s = stream.Stream()
        n, selected_pitch, selected_duration = generate_random_note()
        s.append(n)
        filename = f'{output_folder}/note_{selected_pitch}_{selected_duration}.musicxml'
        s.write('musicxml', fp=filename)

        synthetic_data.append(s)

    return synthetic_data


## PNG Images

In [2]:
import subprocess
import platform
import os


# get the right path for musescore based on system
def get_musescore_path():
    system = platform.system()
    if system == 'Windows':
        return r'C:\Program Files\MuseScore 4\bin\MuseScore4.exe'  # Update this path if necessary
    elif system == 'Darwin':  # macOS
        return '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
    elif system == 'Linux':
        return '/usr/bin/musescore4'  # Update this path if necessary
    else:
        raise ValueError("Unsupported operating system")


def convert_musicxml_to_png(input_folder='../raw_data/musicxml_files', output_folder='../raw_data/sheet_images'):
    current_dir = os.getcwd()
    input_folder = os.path.abspath(os.path.join(current_dir, input_folder))
    output_folder = os.path.abspath(os.path.join(current_dir, output_folder))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    musescore_path = get_musescore_path()

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.musicxml'):
            input_path = os.path.join(input_folder, file_name)
            output_filename = file_name.replace('.musicxml', '.png')
            output_path = os.path.join(output_folder, output_filename)
            result = subprocess.run([musescore_path, input_path, '-o', output_path], stderr=subprocess.PIPE)
            if result.returncode != 0:
                print(f"Error processing {file_name}: {result.stderr.decode('utf-8')}")

            # Check if the file has a '-1' suffix and rename it
            generated_filename = output_filename.replace('.png', '-1.png')
            generated_path = os.path.join(output_folder, generated_filename)
            if os.path.exists(generated_path):
                os.rename(generated_path, output_path)

    return None

# Convert Image and Embed Labels

## For rows of music, to be tried later

In [3]:
import cv2
import numpy as np

#### This will need to be adjusted (img --> file) #####

def create_dataset(num_samples):
    images = []
    labels = []
    for i in range(num_samples):
        img = cv2.imread(f'random_sample_{i}.png', cv2.IMREAD_GRAYSCALE)
        img_array = np.array(img)

        # Example bounding box creation (this should be based on actual note positions)
        bounding_boxes = [(50, 50, 100, 100)]  # Placeholder
        label = ['C4']  # Placeholder

        images.append(img_array)
        labels.append((bounding_boxes, label))

    return np.array(images), labels

## For single note files

In [ ]:
import cv2
import numpy as np

def process_single_note_image(image_path, label):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_array = np.array(img)

    # As we know it's a single note, let's assume the bounding box covers most of the image
    h, w = img_array.shape
    bounding_box = [0, 0, w, h]

    return img_array, bounding_box, label

def create_single_note_dataset(image_folder='../raw_data/sheet_images', label_data='../raw_data/musicxml_files'):
    images = []
    bounding_boxes = []
    labels = []

    for file_name in os.listdir(image_folder):
        if file_name.endswith('.png'):
            note_info = file_name.replace('.png', '').split('_')[1:]  # Extract pitch and duration from filename
            label = '_'.join(note_info)

            img_array, bounding_box, label = process_single_note_image(os.path.join(image_folder, file_name), label)
            images.append(img_array)
            bounding_boxes.append(bounding_box)
            labels.append(label)

    return np.array(images), bounding_boxes, labels

# Preprocess

In [4]:
# grayscale, thresholding, resizing for model

def preprocess_image(image):
    _, thresh_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    resized_image = cv2.resize(thresh_image, (128, 128))
    return resized_image

# Model

## Structure

### Regression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def train_logistic_regression(X, y):
    X_flat = X.reshape(X.shape[0], -1)
    X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    return accuracy

### CNN

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_cnn(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = create_cnn_model(X_train.shape[1:], num_classes=10)  # Example with 10 classes
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    return model.evaluate(X_test, y_test)


In [ ]:
# for running the model, don't run this cell until others are good

images, labels = create_dataset(100)  # Generate 100 samples
preprocessed_images = np.array([preprocess_image(img) for img in images])

# Logistic Regression
logistic_accuracy = train_logistic_regression(preprocessed_images, labels)

# CNN
cnn_accuracy = train_cnn(preprocessed_images, labels)